In [1]:
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
import nltk
import numpy as np
import pandas as pd
from twitter_preprocessor import TwitterPreprocessor
import os
import glob
from collections import Counter

In [2]:
# preperation
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
nltk.download('stopwords')
stopset = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /Users/cary/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cary/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/cary/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/cary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
wordFrqSol= {}
results = {}

In [4]:
# file list
soldierCSVs = []
files = os.listdir(os.getcwd() + "/data/civilians")
for file in files:
     if not os.path.isdir(file) and file.endswith(".csv"):
            soldierCSVs.append(file)

In [5]:
# useless columns for analyzing
uselessColumns = [
    "id",
    "conversation_id",
    "place",
    "photos",
    "video",
    "near",
    "geo",
    "source",
    "user_rt_id",
    "user_rt",
    "retweet_id",
    "reply_to",
    "retweet_date",
    "translate",
    "trans_src",
    "trans_dest",
    "link"
]

In [6]:
def calFrqForTweet(tweet):
    tempTweet = tweet
    tempTweetCleaned = TwitterPreprocessor(tempTweet).remove_urls().remove_mentions().remove_hashtags().remove_twitter_reserved_words().remove_single_letter_words().remove_numbers(preserve_years=True).remove_blank_spaces().text    
    frq = {}
    
    if len(tempTweetCleaned) <= 0:
        return 0, frq
    
    tempTokens = nltk.word_tokenize(tempTweetCleaned)
#     tempTokens = [tempToken.lower() for tempToken in tempTokens if len(tempToken)>2]
#     tempTokens = [tempToken.lower() for tempToken in tempTokens if tempToken.lower() not in stopset and len(tempToken)>2]
    tempTagged = nltk.pos_tag(tempTokens)
    
    count = 0
    for w, p in tempTagged:
        count += 1
        if p.startswith('J'):
            key = w
            if key in frq:
                frq[key] += 1
            else:
                frq[key] = 1
    
    return count, frq

In [7]:
def genTable(res, frqList):
    print(frqList)
    num = 0
    for w, c in frqList:
        num += 1
        key = 'list-' + str(num)
        res[key + '-w'] = w
        res[key + '-f'] = c * 1.0 / res['ttlLen']
    
    return res

In [8]:
for soldierCSVFile in soldierCSVs:
    if soldierCSVFile == 'cleaned_vet_tweet_df.csv': continue
    if soldierCSVFile == 'combined_civilians.csv': continue

    print('Calculating... ' + soldierCSVFile)
    
    tempDF = pd.read_csv(os.getcwd() + "/data/civilians/" + soldierCSVFile, encoding='utf8')
    tempDF.dropna(subset=['tweet'], inplace=True) # clear empty tweets
    tempDF.drop(uselessColumns, axis=1, inplace=True)
    
    singleFRQ = {}
    ttlLen = 0
    tCount = 0

    for id, row in tempDF.iterrows():
        count, freq = calFrqForTweet(row['tweet'])
        singleFRQ = dict(Counter(singleFRQ) + Counter(freq))
        ttlLen += count
        tCount += 1
        if (id % 1000 ) == 0: print(id)
    
    wordFrqSol = dict(Counter(singleFRQ) + Counter(wordFrqSol))
    
    frqList = sorted(singleFRQ.items(), key = lambda x: x[1], reverse=True)[:10]
        
    res = {
        'count': tCount,
        'ttlLen': ttlLen,
    }
    
    res = genTable(res, frqList)
    
    results[soldierCSVFile] = res
    
    print(res)
    

Calculating... myrna99.csv
0
[('limited', 43), ('next', 34), ('other', 26), ('Happy', 25), ('’', 16), ('sorry', 12), ('good', 10), ('great', 8), ('cute', 8), ('best', 7)]
{'count': 940, 'ttlLen': 9301, 'list-1-w': 'limited', 'list-1-f': 0.004623158800129018, 'list-2-w': 'next', 'list-2-f': 0.0036555209117299214, 'list-3-w': 'other', 'list-3-f': 0.0027953983442640575, 'list-4-w': 'Happy', 'list-4-f': 0.0026878830233308245, 'list-5-w': '’', 'list-5-f': 0.0017202451349317277, 'list-6-w': 'sorry', 'list-6-f': 0.0012901838511987958, 'list-7-w': 'good', 'list-7-f': 0.0010751532093323298, 'list-8-w': 'great', 'list-8-f': 0.0008601225674658639, 'list-9-w': 'cute', 'list-9-f': 0.0008601225674658639, 'list-10-w': 'best', 'list-10-f': 0.0007526072465326309}
.csvulating... KBDeSalvo
0
1000
[('great', 134), ('public', 65), ('more', 30), ('new', 29), ('Happy', 28), ('proud', 27), ('impoant', 24), ('healthy', 23), ('Such', 23), ('social', 22)]
{'count': 1659, 'ttlLen': 22665, 'list-1-w': 'great', 'li

/Users/cary/.local/share/virtualenvs/TextAnalytics-9hFs7Vd6/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
[('new', 1189), ('good', 622), ('live', 454), ('next', 378), ('first', 373), ('more', 371), ('awesome', 348), ('great', 347), ('ready', 344), ('much', 326)]
{'count': 44534, 'ttlLen': 416042, 'list-1-w': 'new', 'list-1-f': 0.002857884540503122, 'list-2-w': 'good', 'list-2-f': 0.001495041366015931, 'list-3-w': 'live', 'list-3-f': 0.0010912359809826893, 'list-4-w': 'next', 'list-4-f': 0.0009085621163247941, 'list-5-w': 'first', 'list-5-f': 0.0008965440989130905, 'list-6-w': 'more', 'list-6-f': 0.0008917368919484091, 'list-7-w': 'awesome', 'list-7-f': 0.0008364540118545724, 'list-8-w': 'great', 'list-8-f': 0.0008340504083722317, 'list-9-w': 'ready', 'list-9-f': 0.0008268395979252095, 'list-10-w': 'much', 'list-10-f': 0.00078357473524307

/Users/cary/.local/share/virtualenvs/TextAnalytics-9hFs7Vd6/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
[('good', 3174), ('many', 1461), ('last', 1064), ('great', 1037), ('more', 1015), ('best', 928), ('much', 858), ('big', 840), ('first', 661), ('new', 625)]
{'count': 40986, 'ttlLen': 759032, 'list-1-w': 'good', 'list-1-f': 0.004181641880711222, 'list-2-w': 'many', 'list-2-f': 0.0019248200339379632, 'list-3-w': 'last', 'list-3-f': 0.0014017854319712476, 'list-4-w': 'great', 'list-4-f': 0.001366213809167466, 'list-5-w': 'more', 'list-5-f': 0.00133722952391994, 'list-6-w': 'best', 'list-6-f': 0.0012226098504410881, 'list-7-w': 'much', 'list-7-f': 0.0011303871246535061, 'list-8-w': 'big', 'list-8-f': 0.0011066727094509849, 'list-9-w': 'first', 'list-9-f': 0.0008708460249370251, 'list-10-w': 'new', 'list-10-f': 0.0008234171945319829}
Calculating... carioke.csv
0

In [13]:
# pd.DataFrame.from_dict(results, orient="index").to_csv("soldierFreqCal.csv")
pd.DataFrame.from_dict(results, orient="index").to_csv("civilianFreqCal.csv")

In [10]:
topList = sorted(wordFrqSol.items(), key = lambda x: x[1], reverse=True)[:50]

In [11]:
topList

[('new', 15100),
 ('good', 14266),
 ('great', 11887),
 ('more', 9939),
 ('last', 8760),
 ('best', 6906),
 ('many', 6373),
 ('much', 5863),
 ('next', 5556),
 ('first', 5400),
 ('More', 5391),
 ('other', 5157),
 ('’', 4661),
 ('big', 4571),
 ('Happy', 4267),
 ('happy', 4041),
 ('Good', 3950),
 ('sure', 3886),
 ('better', 3597),
 ('same', 3534),
 ('bad', 3388),
 ('few', 3377),
 ('old', 3179),
 ('little', 3142),
 ('such', 3021),
 ('free', 2949),
 ('real', 2927),
 ('right', 2771),
 ('nice', 2675),
 ('true', 2566),
 ('own', 2523),
 ('ready', 2476),
 ('>', 2455),
 ('live', 2417),
 ('awesome', 2361),
 ('..', 2334),
 ('special', 2333),
 ('beautiful', 2200),
 ('“', 2195),
 ('different', 2106),
 ('full', 2000),
 ('hard', 1982),
 ('latest', 1965),
 ('orite', 1932),
 ('…', 1928),
 ('wrong', 1919),
 ('wonderful', 1918),
 ('long', 1895),
 ('Indian', 1891),
 ('least', 1860)]

In [12]:
results

{'myrna99.csv': {'count': 940,
  'ttlLen': 9301,
  'list-1-w': 'limited',
  'list-1-f': 0.004623158800129018,
  'list-2-w': 'next',
  'list-2-f': 0.0036555209117299214,
  'list-3-w': 'other',
  'list-3-f': 0.0027953983442640575,
  'list-4-w': 'Happy',
  'list-4-f': 0.0026878830233308245,
  'list-5-w': '’',
  'list-5-f': 0.0017202451349317277,
  'list-6-w': 'sorry',
  'list-6-f': 0.0012901838511987958,
  'list-7-w': 'good',
  'list-7-f': 0.0010751532093323298,
  'list-8-w': 'great',
  'list-8-f': 0.0008601225674658639,
  'list-9-w': 'cute',
  'list-9-f': 0.0008601225674658639,
  'list-10-w': 'best',
  'list-10-f': 0.0007526072465326309},
 'KBDeSalvo\r.csv': {'count': 1659,
  'ttlLen': 22665,
  'list-1-w': 'great',
  'list-1-f': 0.005912199426428414,
  'list-2-w': 'public',
  'list-2-f': 0.002867857930730201,
  'list-3-w': 'more',
  'list-3-f': 0.0013236267372600927,
  'list-4-w': 'new',
  'list-4-f': 0.0012795058460180896,
  'list-5-w': 'Happy',
  'list-5-f': 0.0012353849547760865,
  'l